In [12]:
import json
import os

src_filename = "../narrative/rating_DE_EA_spa-3486596_zho.json"
dest_filename = "rating_DE_EA_spa-3486596_zho.csv"
with open(src_filename, "r") as f:
    data = f.read()
    articles = json.loads(data)
    # save to csv file
    if os.path.exists(dest_filename):
        os.remove(dest_filename)
    with open(dest_filename, "w") as f:
        f.write("country,rating\n")
        for article in articles:
            f.write(
                "{},{}\n".format(
                    article["source"]["location"]["label"]["eng"], article["rating"]
                )
            )

In [17]:
import json
import os

src_filename = "../narrative/rating_DE_EA_spa-3486596_eng.json"
dest_filename = "rating_DE_EA_spa-3486596_eng.csv"
with open(src_filename, "r") as f:
    data = f.read()
    articles = json.loads(data)
    # save to csv file
    if os.path.exists(dest_filename):
        os.remove(dest_filename)
    with open(dest_filename, "w") as f:
        f.write("country,rating\n")
        for article in articles:
            # if key exists
            if "country" in article["source"]["location"]:
                f.write(
                    "{},{}\n".format(
                        article["source"]["location"]["country"]["label"]["eng"],
                        article["rating"],
                    )
                )
            else:
                f.write(
                    "{},{}\n".format(
                        article["source"]["location"]["label"]["eng"], article["rating"]
                    )
                )

In [18]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)  # This loads the environment variables from .env file
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [76]:
def QnA(
    art,
    client,
    model="gpt-4o",
    QnA_prompt="Which of the following is correct tone that this article discuss about Raisi's role in Iran's development:",
    QnA_answer="A) Positive \nB) Negative \nC) Neutral \nD) Not applicable",
):

    prompt_complete = (
        QnA_prompt
        + "\n"
        + art["title"]
        + "\n"
        + art["body"]
        + "\n"
        + "Please choose from the options below:"
        + QnA_answer
    )
    model = model
    messages = [{"role": "user", "content": prompt_complete}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [21]:
src_filename = "../narrative/rating_DE_EA_spa-3486596_eng.json"
des_filename = "QnA_DE_EA_spa-3486596_eng.json"
with open(src_filename, "r") as f:
    data = f.read()
    des_articles = []
    src_articles = json.loads(data)
    for src_art in src_articles:
        src_art["QnA"] = QnA(src_art, client, model="gpt-4o")

        des_articles.append(src_art)

if os.path.exists(des_filename):
    os.remove(des_filename)
with open(des_filename, "w") as f:
    json.dump(des_articles, f, indent=4)

In [23]:
def add_QnA(src_filename, des_filename, client, model="gpt-4o"):
    with open(src_filename, "r") as f:
        data = f.read()
        des_articles = []
        src_articles = json.loads(data)
        for src_art in src_articles:
            src_art["QnA"] = QnA(src_art, client, model="gpt-4o")

            des_articles.append(src_art)

    if os.path.exists(des_filename):
        os.remove(des_filename)
    with open(des_filename, "w") as f:
        json.dump(des_articles, f, indent=4)

In [24]:
src_filename = "../narrative/rating_DE_EA_spa-3486596_zho.json"
des_filename = "QnA_DE_EA_spa-3486596_zho.json"
add_QnA(src_filename, des_filename, client, model="gpt-4o")

In [45]:
keywords = "Iran's president"
q = QueryEventsIter(keywords=keywords, lang="zho")
for event in q.execQuery(er, sortBy="size", maxItems=5):
    print(json.dumps(event, indent=4))

{
    "uri": "eng-9600553",
    "concepts": [
        {
            "uri": "http://en.wikipedia.org/wiki/Israel",
            "type": "loc",
            "score": 100,
            "label": {
                "eng": "Israel"
            },
            "location": {
                "type": "country",
                "label": {
                    "eng": "Israel"
                }
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/Rafah",
            "type": "loc",
            "score": 82,
            "label": {
                "eng": "Rafah"
            },
            "location": {
                "type": "place",
                "label": {
                    "eng": "Rafah"
                },
                "country": {
                    "type": "country",
                    "label": {
                        "eng": "Palestine"
                    }
                }
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/Ha

In [37]:
# get global articles
from eventregistry import *
from eventregistry import EventRegistry

load_dotenv()  # This loads the environment variables from .env file
api_key = os.getenv("EVENT_REGISTRY_API_KEY")
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)

keywords = "Iran's president death"
lang = "eng"
dateStart = "2024-05-20"
dateEnd = "2024-05-21"
q = QueryArticlesIter(
    keywords=keywords, lang=lang, dateStart=dateStart, dateEnd=dateEnd
)

filename = dateStart + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="rel",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, indent=4)

In [38]:
keywords = "Iran's president death"
lang = "hin"
dateStart = "2024-05-19"
dateEnd = "2024-05-19"
q = QueryArticlesIter(
    keywords=keywords, lang=lang, dateStart=dateStart, dateEnd=dateEnd
)

filename = dateStart + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="rel",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, indent=4)

In [56]:
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)
# keywords = "Iran's president"
lang = "zho"
dateStart = "2024-05-19"
dateEnd = "2024-05-20"
eventUri = "zho-1619666"
q = QueryEventArticlesIter(
    eventUri=eventUri, lang=lang, dateStart=dateStart, dateEnd=dateEnd
)

filename = dateStart + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="rel",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [57]:
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)
# keywords = "Iran's president"
lang = "hin"
dateStart = "2024-05-19"
dateEnd = "2024-05-20"
eventUri = "hin-362909"
q = QueryEventArticlesIter(
    eventUri=eventUri, lang=lang, dateStart=dateStart, dateEnd=dateEnd
)

filename = dateStart + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="rel",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [60]:
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)
# keywords = "Iran's president"
lang = "eng"
dateStart = "2024-05-19"
dateEnd = "2024-05-20"
eventUri = "eng-9581242"
q = QueryEventArticlesIter(
    eventUri=eventUri, lang=lang, dateStart=dateStart, dateEnd=dateEnd
)

filename = dateStart + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="rel",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [69]:
def QnA(
    art,
    client,
    model,
    QnA_prompt,
    QnA_answer,
):

    prompt_complete = (
        QnA_prompt
        + "\n"
        + art["title"]
        + "\n"
        + art["body"]
        + "\n"
        + "Please choose from the options below:"
        + QnA_answer
    )
    model = model
    messages = [{"role": "user", "content": prompt_complete}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [154]:
def get_LLM_QnA(
    src_filename,
    client,
    model="gpt-4o",
    QnA_prompt="Which of the following is correct tone that this article discuss about Raisi's reputation:",
    QnA_answer="A) Positive\nB) Neutral\nC) Negative\nD) Not applicable\n",
):
    with open(src_filename, "r") as f:
        data = f.read()
        des_articles = []
        src_articles = json.loads(data)
        for src_art in src_articles:
            src_art["Question"] = QnA_prompt
            src_art["Answers"] = QnA_answer
            src_art["LLM_answer"] = QnA(
                src_art,
                client,
                model=model,
                QnA_prompt=QnA_prompt,
                QnA_answer=QnA_answer,
            )

            des_articles.append(src_art)
    des_filename = "QnA_" + src_filename
    if os.path.exists(des_filename):
        os.remove(des_filename)
    with open(des_filename, "w") as f:
        json.dump(des_articles, f, ensure_ascii=False, indent=4)

In [151]:
def add_LLM_QnA(
    src_filename,
    client,
    model,
    QnA_prompt,
    QnA_answer,
):
    with open(src_filename, "r") as f:
        data = f.read()
        des_articles = []
        src_articles = json.loads(data)
        for src_art in src_articles:
            qna = {}
            qna["Question"] = QnA_prompt
            qna["Answers"] = QnA_answer
            qna["LLM_answer"] = QnA(
                src_art,
                client,
                model=model,
                QnA_prompt=QnA_prompt,
                QnA_answer=QnA_answer,
            )
            match = re.search(r"\s*([A-D])\)", qna["LLM_answer"])
            if match:
                qna["post_LLM_answer"] = match.group(1)
            else:
                qna["post_LLM_answer"] = "missing"

            if "QnA" in src_art:
                src_art["QnA"].append(qna)
            else:
                src_art["QnA"] = [qna]

            des_articles.append(src_art)
            # overwrite the file
    with open(src_filename, "w") as f:
        json.dump(des_articles, f, ensure_ascii=False, indent=4)

In [ ]:
import re


def extract_answer(qna_text):
    # Use a regular expression to find the answer choice (A, B, C, or D) at the start of the QnA field
    match = re.search(r"\s*([A-D])\)", qna_text)
    if match:
        return match.group(1)
    return None


def extract_answer_from_json(src_filename):
    # Load the JSON data
    with open(src_filename, "r") as file:
        data = json.load(file)

    # Iterate through the records and extract answers
    for record in data:
        qna_text = record.get("QnA", "")
        answer = extract_answer(qna_text)
        if answer:
            print(f"Extracted answer: {answer}")
        else:
            print("No answer extracted")

# Repeat below to test question and answer

In [149]:
question = "Which of the following is correct tone that this article discuss about Raisi's reputation:"
answer = "A) Positive B) Neutral C) Negative D) Not applicable"
get_LLM_QnA(
    "2024-05-19_hin.json",
    client,
    model="gpt-4o",
    QnA_prompt=question,
    QnA_answer=answer,
)

In [146]:
extract_answer_from_json("QnA_2024-05-19_eng.json")

Extracted answer: C
Extracted answer: B
Extracted answer: C
Extracted answer: B
Extracted answer: B
Extracted answer: B
Extracted answer: C
Extracted answer: C
Extracted answer: B
Extracted answer: A


In [148]:
extract_answer_from_json("QnA_2024-05-19_zho.json")

Extracted answer: B
Extracted answer: B
Extracted answer: B
Extracted answer: B
Extracted answer: B
Extracted answer: B
Extracted answer: A
Extracted answer: C
Extracted answer: B
Extracted answer: B


In [150]:
extract_answer_from_json("QnA_2024-05-19_hin.json")

Extracted answer: C
Extracted answer: C
Extracted answer: B
Extracted answer: B
Extracted answer: B
Extracted answer: C
Extracted answer: B
Extracted answer: B
Extracted answer: C
Extracted answer: C


# Save question and answer after test

In [144]:
import csv

# Q and A to be added as a new row
new_row = [f"{question}", f"{answer}"]

# name of the csv file
filename = "QnA.csv"

# writing to csv file
with open(filename, "a") as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the data row
    csvwriter.writerow(new_row)

In [152]:
question = "Does the article indicate that the cause of the helicopter crash could be bad weather:"
answer = "A) Yes B) No"

In [153]:
add_LLM_QnA(
    "LLM_2024-05-19_eng.json",
    client,
    model="gpt-4o",
    QnA_prompt=question,
    QnA_answer=answer,
)